# Intro

Testing environment for multi-output and vector output GP's

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pymc as pm
import matplotlib.pyplot as plt
from aesara.tensor.nnet import softmax
from aesara.tensor import reshape
import aesara.tensor as at

In [2]:
raw_df = pd.read_excel('Συγκεντρωτικό Ελαιόλαδου.xlsx')
raw_df.head()

,gid,tofid,dname,zn,loc,rvar,uvar,ucul,kol,kor,...,phos,bor,kal.1,mag,ca,zn.1,fe,cu,mn,salt
0,1043,1043KAMB_GA4_01_25728,1043ΚΑΜΒ,6.0,LESVOS,K50AD50,0,0,50,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1044,1044ELKP_GE8_01_25764,1044ΕΛΚΠ,7.0,LESVOS,K70AD30,0,0,70,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1066,1066ELKP_GB8_01_25861,1066ΕΛΚΠ,1.0,LESVOS,K60ADR40,0,0,60,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1074,1074KAGK_GC1_01_25862,1074ΚΑΓΚ,1.0,LESVOS,K50ADR50,0,0,50,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1087,1087_RE7_01_26470,1087ΕΛΣΓ,1.0,LESVOS,KOLOVI,0,0,100,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.concat([raw_df.iloc[:,57:67], raw_df.loc[:, ['loc']]], axis=1)
# df = df.drop(['phph', 'pph', 'dtoc', 'ouv', 'eryth'], axis=1)

In [4]:
df=df[~df.isna().any(axis=1)]
df.head(2)

,hydrdecarboleuagl,hydrmetholeuagl,hydroleuagl,acpin,apig,elenac,eriodic,hydrelacid,hydrtyr,hydrtyrac,loc
0,0.28,2.78,0.12,12.8,2.34,0.38,3.346423,0.0,0.937861,6.52,LESVOS
1,1.04,4.65,0.14,34.7,2.74,0.38,21.686350,0.0,1.561904,16.00,LESVOS


In [5]:
locs= df.loc[:, 'loc'].unique()
locdict= {k:v for k, v in zip(locs, range(1,len(locs)+1))}
df = df.replace({'loc':locdict})
df

,hydrdecarboleuagl,hydrmetholeuagl,hydroleuagl,acpin,apig,elenac,eriodic,hydrelacid,hydrtyr,hydrtyrac,loc
0,0.280000,2.780000,0.120000,12.800000,2.340000,0.380000,3.346423,0.000000,0.937861,6.520000,1
1,1.040000,4.650000,0.140000,34.700000,2.740000,0.380000,21.686350,0.000000,1.561904,16.000000,1
2,0.140000,1.230000,0.180000,29.300000,2.770000,0.090000,3.776513,0.020000,0.164824,2.550000,1
3,0.280000,3.560000,0.120000,24.500000,2.450000,0.050000,9.695694,0.000000,0.484496,5.470000,1
4,0.370000,2.820000,0.290000,20.300000,2.580000,0.260000,19.730404,0.000000,0.266317,4.480000,1
...,...,...,...,...,...,...,...,...,...,...,...
530,1.860902,0.238083,0.204243,5.092014,0.610923,29.723695,0.459046,0.112005,3.082667,1.274567,5
531,0.024550,1.146114,0.522781,3.816603,0.308362,1.386795,0.046392,0.115379,3.731386,0.198419,5
532,0.017652,1.396886,0.350142,4.239429,0.741855,0.827520,0.089590,0.036676,1.670097,0.271215,5
533,0.043756,0.941640,0.270846,4.024584,0.658550,1.503360,0.080616,0.084807,1.055390,0.231182,5


In [6]:
df_hot=pd.get_dummies(df, columns=['loc'])
vectordf = df_hot.iloc[:,-5:]
vectordf
df.iloc[1:,-1].values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [7]:
arr = np.ndarray((1,df.shape[1]))
K = vectordf.shape[1]
for idx, row in vectordf.iterrows():
    # row of distributions
    r = row.values[:,None]
    indicator_row = df.iloc[idx,:-1].values[:,None].T
    copiedrows = np.vstack([indicator_row]*K)
    joined = np.hstack([copiedrows, r])
    arr = np.vstack([arr,joined])


In [8]:
expdf=pd.DataFrame(data = arr[1:,:], columns=df.columns.tolist())

In [9]:
with pm.Model() as model:
    l = pm.HalfNormal('l', 15, shape=11)
    cov = pm.gp.cov.ExpQuad(11, l)
    GP = pm.MvNormal('GP', mu=np.zeros(expdf.shape[0]), cov=cov.full(expdf.values))
    p = pm.Deterministic('p', softmax(at.reshape(GP, (expdf.shape[0]//5,5)), axis=1))
    y = pm.Categorical('y',p=p[1:,:], observed=df.loc[1:,'loc'].values)

In [10]:
with model:
    psamples = pm.sample_prior_predictive(samples=1, random_seed=44)

Sampling: [GP, l, y]


In [11]:
prior = psamples.prior.stack(samples=["chain", "draw"])
prior['p']

<xarray.DataArray 'p' (p_dim_0: 535, p_dim_1: 5, samples: 1)>
array([[[0.19653513],
        [0.20086618],
        [0.20086625],
        [0.20086623],
        [0.2008662 ]],

       [[0.19538924],
        [0.2011527 ],
        [0.20115269],
        [0.20115266],
        [0.20115271]],

       [[0.19182091],
        [0.20204478],
        [0.20204477],
        [0.20204476],
        [0.20204478]],

       ...,

       [[0.20106387],
        [0.20106392],
        [0.20106388],
        [0.20106389],
        [0.19574444]],

       [[0.20111562],
        [0.20111565],
        [0.20111561],
        [0.20111562],
        [0.1955375 ]],

       [[0.20112319],
        [0.2011232 ],
        [0.20112321],
        [0.20112318],
        [0.19550722]]])
Coordinates:
  * p_dim_0  (p_dim_0) int64 0 1 2 3 4 5 6 7 ... 527 528 529 530 531 532 533 534
  * p_dim_1  (p_dim_1) int64 0 1 2 3 4
  * samples  (samples) object MultiIndex
  * chain    (samples) int64 0
  * draw     (samples) int64 0

In [ ]:
with model:
    pm.sample(100, chains=2, tune=100)